## Exercício - Titanic
disponível em https://www.kaggle.com/c/titanic/overview

#### Descrição
O naufrágio do RMS Titanic é um dos naufrágios mais famosos da história. Em 15 de abril de 1912, durante sua viagem inaugural, o Titanic afundou após colidir com um iceberg, matando 1502 de 2224 passageiros e tripulantes. Essa tragédia chocou a comunidade internacional e levou a melhores regulamentações de segurança para os navios.

Uma das razões pelas quais o naufrágio levou a tal perda de vidas foi que não havia botes salva-vidas suficientes para os passageiros e a tripulação. Embora houvesse algum elemento de sorte envolvido na sobrevivência do naufrágio, alguns grupos de pessoas eram mais propensos a sobreviver do que outros, como mulheres, crianças e a classe alta.

Nesse desafio, a atividade é que você conclua a análise de que tipo de pessoa provavelmente sobreviverá. Em particular, solicitamos você deve aplicar classificadores de árvores de decisão para esta atividade.


### Descrição do dataset
Os dados foram divididos em dois grupos:

conjunto de treinamento (train.csv)
conjunto de teste (test.csv)
O conjunto de treinamento deve ser usado para criar seus modelos de aprendizado de máquina. Para o conjunto de treinamento, fornecemos o resultado (também conhecido como "verdade do solo") para cada passageiro. Seu modelo será baseado em "recursos", como sexo e classe dos passageiros. Você também pode usar a engenharia de recursos para criar novos recursos.

O conjunto de teste deve ser usado para ver o desempenho do seu modelo em dados não vistos. Para o conjunto de testes, não fornecemos a verdade básica para cada passageiro. É seu trabalho prever esses resultados. Para cada passageiro no conjunto de testes, use o modelo que você treinou para prever se eles sobreviveram ou não ao naufrágio do Titanic.

Também incluímos gender_submission.csv, um conjunto de previsões que supõem que todas e apenas passageiras sobrevivem, como um exemplo da aparência de um arquivo de envio.

### Dicionário de Dados

**Variable -	Definition	- Key**

survival - Survival	- 0 = No, 1 = Yes

pclass -	Ticket class -	1 = 1st, 2 = 2nd, 3 = 3rd

sex -	Sex	

Age	- Age in years	

sibsp -	# of siblings / spouses aboard the Titanic	

parch- # of parents / children aboard the Titanic	

ticket - Ticket number	

fare - Passenger fare	

cabin- Cabin number	

embarked- Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# carregando os dados

In [2]:
trainDF, testDF = pd.read_csv("../../datasets/titanic/train.csv"), pd.read_csv("../../datasets/titanic/test.csv")

# Apresentando os dados

In [3]:
trainDF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7925.0000,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
testDF.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
trainDF.shape, testDF.shape

((891, 12), (418, 11))

In [6]:
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [7]:
testDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


### Analisando os dados

Podemos perceber que Passenger ID e Name, não são itens úteis para predição. mas precisaremos do ID para fazer a submissão do para o dataset de test. mas iremos remover o nome

In [8]:
trainDF.drop(columns=['Name'], inplace=True)
testDF.drop(columns=['Name'], inplace=True)

In [9]:
trainDF.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7925.0000,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
testDF.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S


transformando **Sex** e **Embarkerd** em binário

In [11]:
trainDF = pd.get_dummies(trainDF, columns=['Sex', 'Embarked'], drop_first=True) 

In [12]:
trainDF.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,1,0,1
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,0,0,0
2,3,1,3,26.0,0,0,STON/O2. 3101282,7925.0000,NaN,0,0,1
3,4,1,1,35.0,1,0,113803,53.1000,C123,0,0,1
4,5,0,3,35.0,0,0,373450,8.0500,NaN,1,0,1


A coluna ticket tem duas partes:
  -  Prefixo do ticket: String opcional.
  -  Número do ticket: sempre irá existir, o número nos ajudará a saber quem estava viajando sozinho ou em grupo e qual a proximidade entre as cabines.
  
Logo, iremos subdividir esta string, deixando apenas a parte do número do ticket.

In [13]:
trainDF['NewTicket'] = trainDF['Ticket'].str.replace(' ', '_').str.split('_').str.get(-1)

Não faz sentido, se apagar esta linhas, outras parecem com o mesmo problema. provavelmente é algo que envolva a codificação do CSV.

In [14]:
trainDF.loc[trainDF['NewTicket'] == 'LINE'] 

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Embarked_Q,Embarked_S,NewTicket
179,180,0,3,36.0,0,0,LINE,0.0,NaN,1,0,1,LINE
271,272,1,3,25.0,0,0,LINE,0.0,NaN,1,0,1,LINE
302,303,0,3,19.0,0,0,LINE,0.0,NaN,1,0,1,LINE
597,598,0,3,49.0,0,0,LINE,0.0,NaN,1,0,1,LINE


In [15]:
trainDF.drop(columns=['Ticket'], inplace=True)

Na coluna idade existem existem 177 valores nulos, o que deve ser feito? 

In [16]:
trainDF.corr().style.background_gradient(cmap='viridis')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
PassengerId,1,-0.00500666,-0.035144,0.0368472,-0.0575268,-0.00165201,0.0165966,0.0429389,-0.0336057,0.0221478
Survived,-0.00500666,1,-0.338481,-0.0772211,-0.0353225,0.0816294,0.0531986,-0.543351,0.00365038,-0.15566
Pclass,-0.035144,-0.338481,1,-0.369226,0.0830814,0.0184427,-0.130815,0.1319,0.221009,0.0817202
Age,0.0368472,-0.0772211,-0.369226,1,-0.308247,-0.189119,-0.025304,0.0932536,-0.0224048,-0.0325233
SibSp,-0.0575268,-0.0353225,0.0830814,-0.308247,1,0.414838,0.0798971,-0.114631,-0.0263537,0.0709406
Parch,-0.00165201,0.0816294,0.0184427,-0.189119,0.414838,1,0.0975916,-0.245489,-0.0812281,0.0630362
Fare,0.0165966,0.0531986,-0.130815,-0.025304,0.0798971,0.0975916,1,-0.0965229,-0.028107,-0.146654
Sex_male,0.0429389,-0.543351,0.1319,0.0932536,-0.114631,-0.245489,-0.0965229,1,-0.0741151,0.125722
Embarked_Q,-0.0336057,0.00365038,0.221009,-0.0224048,-0.0263537,-0.0812281,-0.028107,-0.0741151,1,-0.496624
Embarked_S,0.0221478,-0.15566,0.0817202,-0.0325233,0.0709406,0.0630362,-0.146654,0.125722,-0.496624,1


Como não existe muita correlação entre os atibutos numéricos e a idade. irei colocar a média para os valores faltantes.

In [17]:
trainDF['Age'] = trainDF['Age'].replace(np.nan, trainDF['Age'].mode()[0])
trainDF.info() # Como podemos perceber, os valores faltantes de Age foram preenchidos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Cabin          204 non-null object
Sex_male       891 non-null uint8
Embarked_Q     891 non-null uint8
Embarked_S     891 non-null uint8
NewTicket      891 non-null object
dtypes: float64(2), int64(5), object(2), uint8(3)
memory usage: 65.3+ KB


Existem poucos valores na coluna Cabin, neste caso, como existe menos da metade, não a nada melhor do que dropar a coluna.

In [18]:
trainDF.drop(columns=['Cabin'], inplace=True)
trainDF.info() #Como podemos perceber, a coluna Cabin foi deletada

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
Sex_male       891 non-null uint8
Embarked_Q     891 non-null uint8
Embarked_S     891 non-null uint8
NewTicket      891 non-null object
dtypes: float64(2), int64(5), object(1), uint8(3)
memory usage: 58.4+ KB


# Separação dos dados de teste

In [19]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

In [20]:
trainDF.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,NewTicket
0,1,0,3,22.0,1,0,7.2500,1,0,1,21171
1,2,1,1,38.0,1,0,71.2833,0,0,0,17599
2,3,1,3,26.0,0,0,7925.0000,0,0,1,3101282
3,4,1,1,35.0,1,0,53.1000,0,0,1,113803
4,5,0,3,35.0,0,0,8.0500,1,0,1,373450


In [21]:
X, y = trainDF.values[:, 2:], trainDF.values[:, 1]

In [22]:
X.shape, y.shape

((891, 9), (891,))

In [23]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
model = DecisionTreeClassifier()

In [24]:
validacao_cruzada = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
validacao_cruzada # Não entendi o porquê deste erro

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'unknown' instead.